# Семинар 4

Сегодня  мы с вами попробуем дообучить многоязычную модель перевода NLLB на малоресурсный марийский язык.

 башкирский язык как звучит

https://huggingface.co/docs/transformers/model_doc/nllb - ссылка на модель

- https://huggingface.co/docs/transformers/model_doc/nllb - ссылка на NLLB
- https://arxiv.org/pdf/2310.09917.pdf - сравение различныйх моделей (mBART, NLLB-200, mPLMs)
- https://arxiv.org/pdf/2309.11668.pdf - дальнейшее развитие NLLB
- https://arxiv.org/pdf/2309.08565.pdf - описание NLLB-200
- https://arxiv.org/pdf/2309.03175.pdf - обзорная статья по нескольким моделям
- https://arxiv.org/pdf/2306.09830.pdf - сранительный анализ
- https://arxiv.org/pdf/2304.04675.pdf - сравнение GPT-4 и NLLB



# 0. Пре-реквизиты

`sentencepiece`, бэкенд для нашего токенайзера (алгоритм для преобразования текста в символы из словаря модели)  
`sacremoses`, пакет, необходимый для предварительной обработки текста, с которым были предварительно обучены модели NLLB.  
`sacrebleu`, пакет для оценки моделей перевода  

In [ ]:
import locale
def gpe(x=None):
    return "UTF-8"
locale.getpreferredencoding = gpe

In [ ]:
!pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu  -q

# 1. Exploring the data

In this section, I try to understand what is the training data that I have, and how suitable it is for fine-tuning a NLLB model.

In [ ]:
from datasets import load_dataset

data = load_dataset('AigizK/bashkir-russian-parallel-corpora', split='train')

In [ ]:
data[223]

In [ ]:
import pandas as pd

In [ ]:
trans_df = pd.DataFrame(data)

In [ ]:
print(trans_df.shape)
print(trans_df.columns)

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
trans_df.sample(3)

In [ ]:
trans_df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

df_test = trans_df[trans_df.corpus=='1000 sentences'].copy()
trans_df = trans_df[trans_df.corpus!='1000 sentences'].sample(100000)

df_train_dev, df_dev = train_test_split(trans_df, test_size=0.01)  # 99000 items, 1000 items
df_train, df_dev = train_test_split(trans_df, test_size=0.01)  # 99000 items, 1000 items

df_train.shape, df_dev.shape, df_test.shape

# 2. How well does the data fit into a NLLB tokenizer?

In [ ]:
from transformers import NllbTokenizer
from tqdm.auto import tqdm, trange

In [ ]:
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

In [ ]:
import re

def word_tokenize(text):
    # a very naive word tokenizer for languages with English-like orthography
    return re.findall('(\w+|[^\w\s])', text)

In [ ]:
smpl = df_train.sample(10000, random_state=1)

smpl['rus_toks'] = smpl.ru.apply(tokenizer.tokenize)
smpl['ba_toks'] = smpl.ba.apply(tokenizer.tokenize)

smpl['rus_words'] = smpl.ru.apply(word_tokenize)
smpl['ba_words'] = smpl.ba.apply(word_tokenize)

In [ ]:
smpl.sample(5)[['ba', 'ba_words', 'ba_toks', 'ru', 'rus_words', 'rus_toks']]

In [ ]:
stats = smpl[['rus_toks', 'ba_toks', 'rus_words', 'ba_words']].applymap(len).describe()
stats

In [ ]:
print(stats.rus_toks['mean'] / stats.rus_words['mean'])
print(stats.ba_toks['mean'] / stats.ba_words['mean'])

Хорошие новости: как для русского, так и для башкирского, токенайзер NLLB, кажется, производит около 2 токенов на слово (более точно, 1.73 и 1.7), что означает, что качество перевода тонко настроенного NLLB может быть приемлемым даже без расширения словаря.

In [ ]:
print(tokenizer.unk_token, tokenizer.unk_token_id)

Ещё одна проверка: как часто в выводе токенайзера для башкирского языка встречается токен `<unk>`? Если это происходит слишком часто, нам нужно как-то это исправить.

In [ ]:
texts_with_unk = [text for text in tqdm(trans_df.ba) if tokenizer.unk_token_id in tokenizer(text).input_ids]
print(len(texts_with_unk))

Тексты получились достаточно грязными, поэтому для улучшения качества работы в первую очередь стоит обратить внимание на очистку данных

In [ ]:
import random
s = random.sample(texts_with_unk, 5)
s

По-видимому, большинство текстов с 32575 неизвестными токенами просто содержат пунктуацию, неизвестную токенайзеру NLLB.

Это потому, что модель NLLB была предварительно обучена на нормализованных текстах. Если мы воспроизведем нормализацию, большинство проблем будет исправлено.

In [ ]:
# this code is adapted from  the Stopes repo of the NLLB team
# https://github.com/facebookresearch/stopes/blob/main/stopes/pipelines/monolingual/monolingual_line_processor.py#L214

import re
import sys
import typing as tp
import unicodedata
from sacremoses import MosesPunctNormalizer


mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
texts_with_unk_normed = [text for text in tqdm(texts_with_unk) if tokenizer.unk_token_id in tokenizer(preproc(text)).input_ids]
print(len(texts_with_unk_normed))

Действительно, после нормализации текстов только малая часть из них содержит неизвестных токенов.  
Это может свидетельствовать о том, что нам не нужно обновлять словарь токенайзера для его использования с башкирским языком. Однако малая часть текстов все еще осталась с неизвестными токенами, поэтому в расширении словаря кроется еще один потенциальный путь к улучшению модели

# 3 (optional). Expanding the vocabulary

# 4. Adding a new language tag to the tokenizer and model

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import NllbTokenizer

In [ ]:
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')
print(len(tokenizer))
print(tokenizer.convert_ids_to_tokens([256202, 256203]))

In [ ]:
def fix_tokenizer(tokenizer, new_lang='ba_Cyrl'):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [ ]:
fix_tokenizer(tokenizer)

In [ ]:
print(tokenizer.convert_ids_to_tokens([256202, 256203, 256204])) # ['zul_Latn', 'ba_Cyrl', '<mask>']
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'ba_Cyrl', '<mask>'])) # [256202, 256203, 256204]
# this is consistent now, wow!

In [ ]:
added_token_id = tokenizer.convert_tokens_to_ids('ba_Cyrl')
similar_lang_id = tokenizer.convert_tokens_to_ids('kir_Cyrl')
print(added_token_id, similar_lang_id)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# moving the embedding for "mask" to its new position
model.model.shared.weight.data[added_token_id+1] = model.model.shared.weight.data[added_token_id]
# initializing new language token with a token of a similar language
model.model.shared.weight.data[added_token_id] = model.model.shared.weight.data[similar_lang_id]

# 5. Preparing the training loop

In [ ]:
import gc
import random
import numpy as np
import torch
from tqdm.auto import tqdm, trange
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [ ]:
model.cuda();

In [ ]:
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)

In [ ]:
batch_size = 16  # 32 already doesn't fit well to 15GB of GPU memory
max_length = 128
warmup_steps = 1_000
training_steps = 57000

In [ ]:
losses = []
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

In [ ]:
LANGS = [('ru', 'rus_Cyrl'), ('ba', 'ba_Cyrl')]

def get_batch_pairs(batch_size, data=df_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

print(get_batch_pairs(1))

In [ ]:
MODEL_SAVE_PATH = './models/nllb-rus-ba-v1'

# 6. The training loop

Тренировка модели занимает на 100k примерах занимает примерно 5 часов, так что запаситесь терпением

In [ ]:
model.train()
x, y, loss = None, None, None
cleanup()

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 1000 == 0:
        print(i, np.mean(losses[-1000:]))

    if i % 1000 == 0 and i > 0:
        model.save_pretrained(MODEL_SAVE_PATH)
        tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
pd.Series(losses).ewm(100).mean().plot();

In [ ]:
def translate(text, src_lang='rus_Cyrl', tgt_lang='eng_Latn', a=16, b=1.5, max_input_length=1024, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    #print(inputs.input_ids.shape[1], result.shape[1])
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
xx, yy, lang1, lang2 = get_batch_pairs(1, data=df_dev)
print(xx)
print(yy)
model.eval()
print(translate(xx[0], lang1, lang2, no_repeat_ngram_size=3, num_beams=5))

In [ ]:
!ls -alsh $MODEL_SAVE_PATH

# 6. Using the model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoConfig
from tqdm.auto import tqdm, trange

In [ ]:
# this code is adapted from  the Stopes repo of the NLLB team
# https://github.com/facebookresearch/stopes/blob/main/stopes/pipelines/monolingual/monolingual_line_processor.py#L214

import re
import sys
import typing as tp
import unicodedata
from sacremoses import MosesPunctNormalizer


mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
def fix_tokenizer(tokenizer, new_lang='ba_Cyrl'):
    """ Add a new language token to the tokenizer vocabulary (this should be done each time after its initialization) """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [ ]:
model_load_name = './models/nllb-rus-ba-v1'
model = AutoModelForSeq2SeqLM.from_pretrained(model_load_name).cuda()
tokenizer = NllbTokenizer.from_pretrained(model_load_name)
fix_tokenizer(tokenizer)

In [ ]:
def translate(text, src_lang='rus_Cyrl', tgt_lang='eng_Latn', a=32, b=3, max_input_length=1024, num_beams=4, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
def batched_translate(texts, batch_size=16, **kwargs):
    """Translate texts in batches of similar length"""
    idxs, texts2 = zip(*sorted(enumerate(texts), key=lambda p: len(p[1]), reverse=True))
    results = []
    for i in trange(0, len(texts2), batch_size):
        results.extend(translate(texts2[i: i+batch_size], **kwargs))
    return [p for i, p in sorted(zip(idxs, results))]

In [ ]:
rus_translated = batched_translate(df_dev.ba, src_lang='ba_Cyrl', tgt_lang='rus_Cyrl')

In [ ]:
df_dev['rus_translated'] = [translate(t, 'ba_Cyrl', 'rus_Cyrl')[0] for t in tqdm(df_dev.ba)]
df_dev['ba_translated'] = [translate(t, 'rus_Cyrl', 'ba_Cyrl')[0] for t in tqdm(df_dev.ru)]

In [ ]:
import sacrebleu
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

In [ ]:
xx, yy = ['течёт холод'], ['несёт холодом']
print(bleu_calc.corpus_score(xx, [yy]))
print(chrf_calc.corpus_score(xx, [yy]))
print(chrf_calc.corpus_score(yy, [xx]))

In [ ]:
print(bleu_calc.corpus_score(df_dev['rus_translated'].tolist(), [df_dev['ru'].tolist()]))
print(chrf_calc.corpus_score(df_dev['rus_translated'].tolist(), [df_dev['ru'].tolist()]))
print(bleu_calc.corpus_score(df_dev['ba_translated'].tolist(), [df_dev['ba'].tolist()]))
print(chrf_calc.corpus_score(df_dev['ba_translated'].tolist(), [df_dev['ba'].tolist()]))

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
df_dev.sample(10, random_state=5)[['ba', 'ru', 'ba_translated', 'rus_translated']]

In [ ]:
print((df_dev.ru == df_dev.rus_translated).mean())
print((df_dev.ba == df_dev.ba_translated).mean())

In [ ]:
!pip install editdistance

In [ ]:
import editdistance

def ed_similarity(text1, text2):
    return max(0, 1 - editdistance.eval(text1, text2) / min(len(text1), len(text2)))

print(ed_similarity('кот', 'собака'))
print(ed_similarity('кот', 'кит'))

In [ ]:
pd.Series([ed_similarity(row.ru, row.rus_translated) for row in df_dev.itertuples()]).describe()

In [ ]:
pd.Series([ed_similarity(row.ba, row.ba_translated) for row in df_dev.itertuples()]).describe()

In [ ]:
df_dev.index.name = "row_id"

In [ ]:
df_dev.to_csv(model_load_name + "/dev_set_translated.tsv", sep="\t")

In [ ]:
def translate(
    text,
    model,
    tokenizer,
    src_lang='rus_Cyrl',
    tgt_lang='ba_Cyrl',
    max_length='auto',
    num_beams=4,
    no_repeat_ngram_size=4,
    n_out=None,
    **kwargs
):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    if max_length == 'auto':
        max_length = int(32 + 2.0 * encoded.input_ids.shape[1])
    model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out

In [ ]:
translate("красная птица", model=model, tokenizer=tokenizer)

Добиться более хорошего качества помимо очистки вавших данных можно за счет расширения словаря модели токенами для вашего целевого языка - это можно сделать если обучить токенизатор с нуля и расширить ваш исходный токенизатор, а позже и модель

Помимо всего прочего, попробуйте посмотреть, как будет отличаться качество генерации при варьировании различных параметров генерации

Для всех моделей самым дешевым методом улучшения качества генерации будет увеличение параметра beam_size для алгоритма beam-search. Однако стоит понимать, что это улучшение качества приведет к замедлению работы вашей модели

Также стоит заметить, что за счет более долгой тренировки можно добиться улучшения на вашем малоресурсном языке, однако качество перевода для русского языка будет немного просаживаться